<a href="https://colab.research.google.com/github/Borg2/Naive-Bayes-for-text-classification/blob/main/NaiveBayes_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("sst", "default")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2210 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 2210
    })
})

function to add class column

In [ ]:
def addClasses(data):
  i=0
  classes=[None]*len(data)
  for label in data['label']:
    if 0<=label <=0.2:
      classes[i]="very negative"
    elif 0.2<label <=0.4:
      classes[i]="negative"
    elif 0.4<label <=0.6:
      classes[i]="neutral"
    elif 0.6<label <=0.8:
      classes[i]="positive"
    elif 0.8<label <=1:
      classes[i]="very positive"
    i+=1
  data=data.add_column("class",classes)
  return data

In [ ]:
train_ds=dataset['train']
valid_ds=dataset['validation']
test_ds=dataset['test']

In [ ]:
train_ds=addClasses(train_ds)
train_ds

Dataset({
    features: ['sentence', 'label', 'tokens', 'tree', 'class'],
    num_rows: 8544
})

Removing unnecessary columns

In [ ]:
train_ds=train_ds.remove_columns(["tokens","tree"])
train_ds

Dataset({
    features: ['sentence', 'label', 'class'],
    num_rows: 8544
})

function to map the index to class

In [ ]:
def classMap(index):
  if index ==0:
    return "very negative"
  elif index == 1:
    return "negative"
  elif index== 2:
    return "neutral"
  elif index == 3:
    return "positive"
  elif index ==4:
    return "very positive"
def keyMap(c):
  if c=="very negative":
    return 0
  elif c=="negative":
    return 1
  elif c=="neutral":
    return 2
  elif c=="positive":
    return 3
  elif c=="very positive":
    return 4

#Naive Bayes

In [ ]:
import numpy as np

class NaiveBayes:

  def Train(self,data,classes,vocab):
    self.logprior = np.zeros(len(classes))
    self.loglikelihood = np.zeros((len(vocab),len(classes)))

    for idx,class_label in enumerate(classes):
      Ndoc=len(data)
      Nc=0
      classDocs=[]
      for d in data:
        if d['class'] == class_label:
          Nc+=1
          classDocs.append(d['sentence'])#adding the sentence


      self.logprior[idx] = np.log(Nc / Ndoc)
      countOfAllWords=0
      #get the count of all the words in class
      for sentence in classDocs:
        words = sentence.split()
        countOfAllWords+=len(words)
      for i,w in enumerate(vocab):
        wordCount = sum(1 for d in classDocs if w in d)#gettin the count of the word in the class
        self.loglikelihood[i][idx] = np.log((wordCount + 1) / (countOfAllWords + len(vocab)))#measure the likelihood for each word with each class

    return self.logprior, self.loglikelihood, vocab
  def predict(self,testDoc,vocab,classes):
    sum=np.zeros(len(classes))
    vocab_list=list(vocab.keys())#changing vocab from a dict to a list
    for idx,class_label in enumerate(classes):
      sum[idx]=self.logprior[idx]
      words=testDoc.split()
      for word in words:
        if word in vocab_list:
          i=vocab_list.index(word)
          sum[idx]+=self.loglikelihood[i][idx]
    return classMap(np.argmax(sum))


Performing count vectorizer on the Dataset

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import string

def custom_tokenizer(text):
    text_no_punctuation = "".join([char for char in text if char not in string.punctuation])
    tokens = text_no_punctuation.split()
    return tokens

count_vectorizer = CountVectorizer(
    stop_words="english",
    token_pattern=r'\b\w\w+\b',
    tokenizer=custom_tokenizer,
    lowercase=True
)

train_text = count_vectorizer.fit(train_ds["sentence"])
vocab=train_text.vocabulary_
print(vocab)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'rock': 11930, 'destined': 3796, '21st': 92, 'century': 2273, 's': 12088, 'new': 9519, 'conan': 2871, 'going': 6085, 'make': 8622, 'splash': 13347, 'greater': 6221, 'arnold': 895, 'schwarzenegger': 12281, 'jeanclaud': 7679, 'van': 15338, 'damme': 3450, 'steven': 13564, 'segal': 12412, 'gorgeously': 6134, 'elaborate': 4528, 'continuation': 3032, 'lord': 8445, 'rings': 11878, 'trilogy': 14731, 'huge': 6902, 'column': 2735, 'words': 15968, 'adequately': 371, 'cowriterdirector': 3194, 'peter': 10391, 'jackson': 7630, 'expanded': 4982, 'vision': 15486, 'jrr': 7772, 'tolkien': 14515, 'middleearth': 8979, 'singercomposer': 12857, 'bryan': 1922, 'adams': 351, 'contributes': 3050, 'slew': 12981, 'songs': 13190, 'potential': 10747, 'hits': 6747, 'simply': 12846, 'intrusive': 7523, 'story': 13630, 'package': 10066, 'certainly': 2278, 'captures': 2135, 'intended': 7441, 'er': 4790, 'spirit': 13335, 'piece': 10451, 'd': 3435, 'think': 14351, 'america': 640, 'plucky': 10585, 'british': 1871, 'eccen

Training the naive bayes model

In [ ]:
Classes=["very negative","negative","neutral","positive","very positive"]
NBmodel=NaiveBayes()
NBmodel.Train(train_ds,Classes,vocab)

Testing the model

In [ ]:
print(NBmodel.predict("this movie was fantastic",vocab,Classes))
print(NBmodel.predict("i didnt like the movie but the cinematography was pretty good",vocab,Classes))

very positive
negative


Adding column class to test dataset

In [ ]:
test_ds=addClasses(test_ds)
test_ds

Dataset({
    features: ['sentence', 'label', 'tokens', 'tree', 'class'],
    num_rows: 2210
})

Testing the model on the test dataset

In [ ]:
from sklearn.metrics import accuracy_score
predictions=[]
for idx,sentence in enumerate(test_ds['sentence']):
  predictions.append(NBmodel.predict(sentence,vocab,Classes))

print("accuracy:",accuracy_score(test_ds['class'],predictions)*100)

accuracy: 38.19004524886878


Testing the accuracy of the sklearn Naive bayes model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(CountVectorizer(),MultinomialNB())
pipeline.fit(train_ds['sentence'],train_ds['class'] )
y_pred = pipeline.predict(test_ds['sentence'])
print("accuracy:",accuracy_score(test_ds['class'],y_pred)*100)

accuracy: 40.90497737556561


function to get the confusion matrix and calculate the metrics

In [ ]:
def getConfusionMatrix(true,pred):
  cm=np.zeros((len(Classes),len(Classes)))
  for idx,_ in enumerate(true):
    cm[keyMap(true[idx])] [keyMap(pred[idx])]+=1
  return cm
def calculateMetrics(cm):
  n=cm.shape[0]
  per=np.zeros(n)
  recall=np.zeros(n)
  f1=np.zeros(n)
  for i in range(0,n):
    per[i]=cm[i][i]/np.sum(cm[:,i])
    recall[i]=cm[i][i]/np.sum(cm[i,:])
    f1[i]=2*(per[i]*recall[i])/(per[i]+recall[i])
  return per,recall,f1


Gettin the confusion matrix for the model

In [ ]:
cm=getConfusionMatrix(test_ds['class'],predictions)
print(cm)

[[ 49. 139.  25.  51.  15.]
 [ 58. 306.  58. 171.  40.]
 [ 14. 132.  40. 164.  39.]
 [ 16.  80.  27. 281. 106.]
 [  5.  40.  11. 175. 168.]]


calculating the metrics for the model

In [ ]:
per,recall,f1=calculateMetrics(cm)
print("percision:",per)
print("recall:",recall)
print("F1 score:",f1)

percision: [0.34507042 0.43902439 0.2484472  0.33372922 0.45652174]
recall: [0.17562724 0.48341232 0.10282776 0.55098039 0.42105263]
F1 score: [0.2327791  0.46015038 0.14545455 0.41568047 0.4380704 ]


Gettin the confusion matrix for the sklearn model

In [ ]:
cm=getConfusionMatrix(test_ds['class'],y_pred)
print(cm)

[[ 28. 200.  10.  41.   0.]
 [ 22. 415.  57. 127.  12.]
 [  8. 154.  34. 181.  12.]
 [  2.  88.  39. 340.  41.]
 [  2.  39.  12. 259.  87.]]


calculating the metrics for the sklearn model

In [ ]:
per,recall,f1=calculateMetrics(cm)
print("percision:",per)
print("recall:",recall)
print("F1 score:",f1)

percision: [0.4516129  0.46316964 0.22368421 0.35864979 0.57236842]
recall: [0.10035842 0.65560821 0.0874036  0.66666667 0.21804511]
F1 score: [0.16422287 0.54283846 0.12569316 0.46639232 0.31578947]
